In [1]:
import os
import h5py
import time
import math
import redis
import ctypes
import numpy as np
import pandas as pd
# from multiprocessing.sharedctypes import RawArray
from multiprocessing.shared_memory import SharedMemory

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [3]:
from libs.quotation import Quotation
from libs.utils import Utils
from libs.tdx import TDX

In [4]:

class DailyData:

    rd = redis.Redis(host='127.0.0.1', port=6379, db=8)

    basics_columns = ['zt_price', 'dt_price', 'ma5vpm',
                      'mcap', 'sum4', 'sum9', 'sum19', 'sum29', 'sum59']
    snapshots_columns = ['open', 'close', 'now', 'high',
                         'low', 'turnover', 'volume', 'bid1', 'bid1_volume']
    statistic_columns = ['zhangfu', 'junjia', 'liangbi', 'zhangsu',
                         'tingban', 'sum5', 'sum10', 'sum20', 'sum30', 'sum60']

    # date: %Y%m%d
    def __init__(self, date=None, symbols=[], check_points=[], create=False):

        if date is None:
            self.date = time.strftime('%Y%m%d')
        else:
            self.date = date

        self.hdf5_file = os.path.join(
            os.getcwd(), 'storage', f'{self.date}.hdf5')

        if create:
            self.rd.set(f'hq_{self.date}_symbols_length', len(symbols))
            self.rd.set(f'hq_{self.date}_check_points_length',
                        len(check_points))

            self.shm_symbols = SharedMemory(
                name=f'${self.date}_symbols', create=True, size=len(symbols)*np.dtype('<U6').itemsize)
            self.shm_names = SharedMemory(name=f'${self.date}_names', create=True, size=len(
                symbols)*np.dtype('<U4').itemsize)
            self.shm_check_points = SharedMemory(
                name=f'${self.date}_check_points', create=True, size=len(check_points)*np.dtype('<u4').itemsize)
            self.shm_basics = SharedMemory(name=f'${self.date}_basics', create=True, size=len(
                check_points)*len(symbols)*len(self.basics_columns)*np.dtype('<f8').itemsize)
            self.shm_snapshots = SharedMemory(name=f'${self.date}_snapshots', create=True, size=len(
                check_points)*len(symbols)*len(self.snapshots_columns)*np.dtype('<f8').itemsize)
            self.shm_statistic = SharedMemory(name=f'${self.date}_statistic', create=True, size=len(
                check_points)*len(symbols)*len(self.statistic_columns)*np.dtype('<f8').itemsize)

            self.symbols = np.ndarray(
                (len(symbols),), dtype='<U6', buffer=self.shm_symbols.buf)
            self.names = np.ndarray(
                (len(symbols),), dtype='<U4', buffer=self.shm_names.buf)
            self.check_points = np.ndarray(
                (len(check_points),),
                dtype='<u4',
                buffer=self.shm_check_points.buf
            )

        else:
            self.shm_symbols = SharedMemory(name=f'${self.date}_symbols')
            self.shm_names = SharedMemory(name=f'${self.date}_names')
            self.shm_check_points = SharedMemory(
                name=f'${self.date}_check_points')
            self.shm_basics = SharedMemory(name=f'${self.date}_basics')
            self.shm_snapshots = SharedMemory(name=f'${self.date}_snapshots')
            self.shm_statistic = SharedMemory(name=f'${self.date}_statistic')

            symbols_length = int(self.rd.get(f'hq_{self.date}_symbols_length'))
            check_points_length = int(self.rd.get(
                f'hq_{self.date}_check_points_length'))

            self.symbols = np.ndarray(
                (symbols_length,), dtype='<U6', buffer=self.shm_symbols.buf)
            self.names = np.ndarray(
                (symbols_length,), dtype='<U4', buffer=self.shm_names.buf)
            self.check_points = np.ndarray(
                (check_points_length,),
                dtype='<u4',
                buffer=self.shm_check_points.buf
            )

            self.post_init()

        self.basics = np.ndarray(
            (len(self.symbols), len(self.basics_columns)),
            dtype='<f8',
            buffer=self.shm_basics.buf
        )
        self.snapshots = np.ndarray(
            (len(self.check_points), len(self.symbols), len(self.snapshots_columns)),
            dtype='<f8',
            buffer=self.shm_snapshots.buf
        )
        self.statistic = np.ndarray(
            (len(self.check_points), len(self.symbols), len(self.statistic_columns)),
            dtype='<f8',
            buffer=self.shm_statistic.buf
        )

    async def prepare(self, symbols, check_points):
        q = Quotation(symbols)
        snapshot = await q.snapshot()

        self.symbols[:] = symbols[:]
        self.names[:] = [snapshot[symbol]['name'] for symbol in symbols][:]
        self.check_points[:] = check_points[:]

        self.basics.fill(np.nan)
        self.snapshots.fill(np.nan)
        self.statistic.fill(np.nan)

        market_values = await q.get_market_values()

        await q.exit()

        tdx = TDX()
        # assert tdx.is_local_tdx_data_outdated() is not True
        klines = tdx.kline(symbols)

        for _, symbol in enumerate(symbols):
            self.basics[_, 0] = market_values[symbol]['zt_price']
            self.basics[_, 1] = market_values[symbol]['dt_price']
            self.basics[_, 3] = market_values[symbol]['mcap']

            if symbol not in klines:
                continue

            kline = klines[symbol]
            self.basics[_, 2] = kline.iloc[0 -
                                           min(5, len(kline)):]['volume'].sum()/1200
            self.basics[_, 4] = kline.iloc[-4:]['close'].sum() if len(kline) >= 4 else np.nan
            self.basics[_, 5] = kline.iloc[-9:]['close'].sum() if len(kline) >= 9 else np.nan
            self.basics[_, 6] = kline.iloc[-19:]['close'].sum() if len(kline) >= 19 else np.nan
            self.basics[_, 7] = kline.iloc[-29:]['close'].sum() if len(kline) >= 29 else np.nan
            self.basics[_, 8] = kline.iloc[-59:]['close'].sum() if len(kline) >= 59 else np.nan

        # await q.exit()
        self.post_init()

    def post_init(self):
        self.active_time_blocks = []
        self.check_interval = self.check_points[2] - self.check_points[1]
        start_time_0_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 09:15:00'
        start_time_1_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 09:30:00'
        start_time_2_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 13:00:00'
        end_time_0_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 09:25:00'
        end_time_1_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 11:30:00'
        end_time_2_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 15:00:00'
        start_time_0 = int(time.mktime(time.strptime(
            start_time_0_str, '%Y-%m-%d %H:%M:%S')))
        start_time_1 = int(time.mktime(time.strptime(
            start_time_1_str, '%Y-%m-%d %H:%M:%S')))
        start_time_2 = int(time.mktime(time.strptime(
            start_time_2_str, '%Y-%m-%d %H:%M:%S')))
        end_time_0 = int(time.mktime(time.strptime(
            end_time_0_str, '%Y-%m-%d %H:%M:%S')))
        end_time_1 = int(time.mktime(time.strptime(
            end_time_1_str, '%Y-%m-%d %H:%M:%S')))
        end_time_2 = int(time.mktime(time.strptime(
            end_time_2_str, '%Y-%m-%d %H:%M:%S')))

        if start_time_0 >= self.check_points[0]:
            start_time_idx_0 = np.where(
                self.check_points == start_time_0)[0][0]
            end_time_idx_0 = np.where(self.check_points == end_time_0)[0][0]

            self.active_time_blocks.append(
                (
                    (start_time_idx_0, start_time_0, start_time_0_str),
                    (end_time_idx_0, end_time_0, end_time_0_str)
                )
            )

        start_time_idx_1 = np.where(self.check_points == start_time_1)[0][0]
        end_time_idx_1 = np.where(self.check_points == end_time_1)[0][0]
        self.active_time_blocks.append(
            (
                (start_time_idx_1, start_time_1, start_time_1_str),
                (end_time_idx_1, end_time_1, end_time_1_str)
            )
        )
        start_time_idx_2 = np.where(self.check_points == start_time_2)[0][0]
        end_time_idx_2 = np.where(self.check_points == end_time_2)[0][0]
        self.active_time_blocks.append(
            (
                (start_time_idx_2, start_time_2, start_time_2_str),
                (end_time_idx_2, end_time_2, end_time_2_str)
            )
        )
        # self.rd.set(f'{self.date}_data_ready', 'true')

    def get_securities(self):
        return pd.DataFrame({
            "symbol": self.symbols,
            "name": self.names,
            "zt_price": self.basics[:, 0],
            "dt_price": self.basics[:, 1],
            "ma5vpm": self.basics[:, 2],
            "mcap": self.basics[:, 3],
            "sum4": self.basics[:, 4],
            "sum9": self.basics[:, 5],
            "sum19": self.basics[:, 6],
            "sum29": self.basics[:, 7],
            "sum59": self.basics[:, 8]
        }).set_index('symbol')

    # check_point: timestamp or %H:%M:%S

    def get_snapshot(self, check_point):
        if type(check_point) == int and check_point <= 100000:
            index = check_point
        elif type(check_point) == float or type(check_point) == int:
            index = np.where(self.check_points == check_point)[0][0]
        elif type(check_point) == str:
            index = [time.strftime('%H:%M:%S', time.localtime(cp))
                     for cp in self.check_points].index(check_point)

        return pd.DataFrame({
            "datetime": [check_point if type(check_point) == str else time.strftime('%H:%M:%S', time.localtime(check_point)) for _ in range(len(self.symbols))],
            "timestamp": [self.check_points[index] if type(check_point) == str else check_point for _ in range(len(self.symbols))],
            "symbol": self.symbols,
            "name": self.names,
            "open": self.snapshots[index, :, 0],
            "close": self.snapshots[index, :, 1],
            "now": self.snapshots[index, :, 2],
            "high": self.snapshots[index, :, 3],
            "low": self.snapshots[index, :, 4],
            "turnover": self.snapshots[index, :, 5],
            "volume": self.snapshots[index, :, 6],
            "bid1": self.snapshots[index, :, 7],
            "bid1_volume": self.snapshots[index, :, 8],
            "zhangfu": self.statistic[index, :, 0],
            "junjia": self.statistic[index, :, 1],
            "liangbi": self.statistic[index, :, 2],
            "zhangsu": self.statistic[index, :, 3],
            "tingban": self.statistic[index, :, 4],
            "ma5": self.statistic[index, :, 5]
        }).set_index('symbol')

    # dt: %Y%m%d
    def save(self, gzip_level=4):
        folder = os.path.join(os.getcwd(), 'storage')
        if not os.path.exists(folder):
            os.mkdir(folder)
#         file = os.path.join(folder, f'{self.date}.hdf5')

        if os.path.exists(self.hdf5_file):
            print('文件 [', self.hdf5_file, '] 已经存在，将被删除 ... ... ', end='')
            os.remove(self.hdf5_file)
            print('已被删除')

        symbols = np.char.encode(self.symbols, encoding='utf-8')
        names = np.char.encode(self.names, encoding='utf-8')
        with h5py.File(self.hdf5_file, "a") as f:
            f.create_dataset(u"symbols", data=symbols,
                             compression="gzip", compression_opts=gzip_level)
            f.create_dataset(u"names", data=names,
                             compression="gzip", compression_opts=gzip_level)
            f.create_dataset(u"check_points", data=self.check_points,
                             compression="gzip", compression_opts=gzip_level)
            f.create_dataset(u"basics", data=self.basics,
                             compression="gzip", compression_opts=gzip_level)
            f.create_dataset(u"snapshots", data=self.snapshots,
                             compression="gzip", compression_opts=gzip_level)
#             f.create_dataset(u"statistic", data=self.statistic, compression="gzip", compression_opts=gzip_level)

        print('行情 已经 写入文件：', self.hdf5_file)

    # dt: %Y%m%d
    @staticmethod
    def load(dt=None):
        date = time.strftime("%Y%m%d", time.localtime()) if dt is None else dt
        file = os.path.join(os.getcwd(), 'storage', f'{date}.hdf5')
        assert os.path.exists(file), 'file['+file+'] does not exists!'

        with h5py.File(file, "a") as f:
            symbols = np.char.decode(f[u'symbols'], 'utf-8')
            check_points = f[u'check_points'][:]

            data = DailyData(date=date, symbols=symbols,
                             check_points=check_points, create=True)
            data.symbols[:] = symbols[:]
            data.names[:] = np.char.decode(f[u'names'], 'utf-8')
            data.check_points[:] = check_points
            data.basics[:, :] = f[u'basics'][:, :]
            data.snapshots[:, :, :] = f[u'snapshots'][:, :, :]
#             data.statistic[:,:,:] = f[u'statistic'][:,:,:]

        data.post_init()
        return data

    def get_ma5pm_anchor_idx(self, idx):
        #         st = time.time()

        ck = self.check_points[idx]

        if ck <= self.active_time_blocks[0][0][1]+300:
            ma5pm_anchor_idx = self.active_time_blocks[0][0][0] - 1

        elif self.active_time_blocks[0][1][1] < ck < self.active_time_blocks[1][0][1]:
            ma5pm_anchor_idx = int(
                self.active_time_blocks[0][1][0] - 300/self.check_interval)

        elif self.active_time_blocks[1][0][1] <= ck <= self.active_time_blocks[1][0][1]+300:
            ma5pm_anchor_idx = self.active_time_blocks[1][0][0] - 1

        elif self.active_time_blocks[1][1][1] < ck < self.active_time_blocks[2][0][1]:
            ma5pm_anchor_idx = int(
                self.active_time_blocks[1][1][0] - 300/self.check_interval)

        elif self.active_time_blocks[2][0][1] <= ck <= self.active_time_blocks[2][0][1]+300:
            result = math.ceil(self.check_points[idx]/60)*60-300
            offset = int((result-self.check_points[idx])/self.check_interval)

            ma5pm_anchor_idx = int(
                max(idx+offset-2, self.active_time_blocks[1][1][0] - 240/self.check_interval))

            if ma5pm_anchor_idx == self.active_time_blocks[1][1][0]:
                ma5pm_anchor_idx += 1

        elif self.active_time_blocks[2][1][1] < ck:
            ma5pm_anchor_idx = int(
                self.active_time_blocks[2][1][0]-300/self.check_interval)

        else:
            result = math.ceil(self.check_points[idx]/60)*60-300
            offset = int((result-self.check_points[idx])/self.check_interval)

            ma5pm_anchor_idx = idx+offset

#         et = time.time()

#         print(
#             idx, ' : ', ma5pm_anchor_idx, '       ',
#             time.strftime("%H:%M:%S", time.localtime(self.check_points[idx])),
#             ' ==> ',
#             time.strftime("%H:%M:%S", time.localtime(self.check_points[ma5pm_anchor_idx])), '       ',
#             ck, ' : ', self.check_points[ma5pm_anchor_idx], '       ',
#             et-st
#         )

        return ma5pm_anchor_idx

    def get_time_lapse(self, idx):

        ck = self.check_points[idx]
        offset = 0
        if ck < self.active_time_blocks[1][0][1]:
            start_time = self.active_time_blocks[0][0][1]
            if ck > self.active_time_blocks[0][1][1]:
                ck = self.active_time_blocks[0][1][1]
        elif ck < self.active_time_blocks[2][0][1]:
            start_time = self.active_time_blocks[1][0][1]
            if ck > self.active_time_blocks[1][1][1]:
                ck = self.active_time_blocks[1][1][1]
        else:
            start_time = self.active_time_blocks[2][0][1]
            if ck > self.active_time_blocks[2][1][1]:
                ck = self.active_time_blocks[2][1][1]
            offset = 120

        return max(int(math.ceil((ck - start_time)/60)), 1)+offset

    def close_sharedmemory(self):
        self.shm_symbols.close()
        self.shm_names.close()
        self.shm_check_points.close()
        self.shm_basics.close()
        self.shm_snapshots.close()
        self.shm_statistic.close()

    def unlink_sharedmemory(self):
        self.shm_symbols.unlink()
        self.shm_names.unlink()
        self.shm_check_points.unlink()
        self.shm_basics.unlink()
        self.shm_snapshots.unlink()
        self.shm_statistic.unlink()

    def incremental_save(self, idx):
        # with h5py.File(self.hdf5_file, "r+") as f:
        #     f[u'snapshots'][idx] = self.snapshots[idx]

        if not hasattr(self, 'hdf5'):
            if not os.path.exists(self.hdf5_file):
                return 'file['+self.hdf5_file+'] does not exists!'
            self.hdf5 = h5py.File(self.hdf5_file, 'r+')

        self.hdf5[u'snapshots'][idx] = self.snapshots[idx]

        if (idx+1) == len(self.check_points):
            self.hdf5.close()
        else:
            self.hdf5.flush()


In [ ]:
from libs.utils import Utils

In [ ]:
Utils.update_symbols()

In [ ]:
snapshot_interval = 5
check_points = Utils.get_check_points()
symbols = Utils.get_running_symbols()

In [ ]:
data = DailyData(symbols, check_points)

In [ ]:
await data.prepare(symbols, check_points)

In [ ]:
data.save()

In [ ]:
data = DailyData.load()

In [ ]:
data.get_securities()

In [ ]:
data.start_points

In [ ]:
'300933' in data.symbols

In [ ]:
data.get_snapshot('09:16:00')

In [ ]:
data.get_snapshot(1611018960)

In [ ]:
data.statistic.shape

In [ ]:
data = DailyData.load(dt='20210812')

In [56]:
########################   测试 get_ma5pm_anchor_idx, get_time_lapse  ####################################
for _, ck in enumerate(data.check_points):
    print(_, ' | ', time.strftime("%H:%M:%S", time.localtime(ck)),
          ' -> ',
          time.strftime("%H:%M:%S", time.localtime(data.check_points[data.get_ma5pm_anchor_idx(_)])), ' | ',
          data.get_time_lapse(_)
         )

0  |  09:14:50  ->  09:14:50  |  1
1  |  09:15:00  ->  09:14:50  |  1
2  |  09:15:05  ->  09:14:50  |  1
3  |  09:15:10  ->  09:14:50  |  1
4  |  09:15:15  ->  09:14:50  |  1
5  |  09:15:20  ->  09:14:50  |  1
6  |  09:15:25  ->  09:14:50  |  1
7  |  09:15:30  ->  09:14:50  |  1
8  |  09:15:35  ->  09:14:50  |  1
9  |  09:15:40  ->  09:14:50  |  1
10  |  09:15:45  ->  09:14:50  |  1
11  |  09:15:50  ->  09:14:50  |  1
12  |  09:15:55  ->  09:14:50  |  1
13  |  09:16:00  ->  09:14:50  |  1
14  |  09:16:05  ->  09:14:50  |  2
15  |  09:16:10  ->  09:14:50  |  2
16  |  09:16:15  ->  09:14:50  |  2
17  |  09:16:20  ->  09:14:50  |  2
18  |  09:16:25  ->  09:14:50  |  2
19  |  09:16:30  ->  09:14:50  |  2
20  |  09:16:35  ->  09:14:50  |  2
21  |  09:16:40  ->  09:14:50  |  2
22  |  09:16:45  ->  09:14:50  |  2
23  |  09:16:50  ->  09:14:50  |  2
24  |  09:16:55  ->  09:14:50  |  2
25  |  09:17:00  ->  09:14:50  |  2
26  |  09:17:05  ->  09:14:50  |  3
27  |  09:17:10  ->  09:14:50  |  3
28

255  |  09:41:00  ->  09:36:00  |  11
256  |  09:41:05  ->  09:37:00  |  12
257  |  09:41:10  ->  09:37:00  |  12
258  |  09:41:15  ->  09:37:00  |  12
259  |  09:41:20  ->  09:37:00  |  12
260  |  09:41:25  ->  09:37:00  |  12
261  |  09:41:30  ->  09:37:00  |  12
262  |  09:41:35  ->  09:37:00  |  12
263  |  09:41:40  ->  09:37:00  |  12
264  |  09:41:45  ->  09:37:00  |  12
265  |  09:41:50  ->  09:37:00  |  12
266  |  09:41:55  ->  09:37:00  |  12
267  |  09:42:00  ->  09:37:00  |  12
268  |  09:42:05  ->  09:38:00  |  13
269  |  09:42:10  ->  09:38:00  |  13
270  |  09:42:15  ->  09:38:00  |  13
271  |  09:42:20  ->  09:38:00  |  13
272  |  09:42:25  ->  09:38:00  |  13
273  |  09:42:30  ->  09:38:00  |  13
274  |  09:42:35  ->  09:38:00  |  13
275  |  09:42:40  ->  09:38:00  |  13
276  |  09:42:45  ->  09:38:00  |  13
277  |  09:42:50  ->  09:38:00  |  13
278  |  09:42:55  ->  09:38:00  |  13
279  |  09:43:00  ->  09:38:00  |  13
280  |  09:43:05  ->  09:39:00  |  14
281  |  09:4

513  |  10:02:30  ->  09:58:00  |  33
514  |  10:02:35  ->  09:58:00  |  33
515  |  10:02:40  ->  09:58:00  |  33
516  |  10:02:45  ->  09:58:00  |  33
517  |  10:02:50  ->  09:58:00  |  33
518  |  10:02:55  ->  09:58:00  |  33
519  |  10:03:00  ->  09:58:00  |  33
520  |  10:03:05  ->  09:59:00  |  34
521  |  10:03:10  ->  09:59:00  |  34
522  |  10:03:15  ->  09:59:00  |  34
523  |  10:03:20  ->  09:59:00  |  34
524  |  10:03:25  ->  09:59:00  |  34
525  |  10:03:30  ->  09:59:00  |  34
526  |  10:03:35  ->  09:59:00  |  34
527  |  10:03:40  ->  09:59:00  |  34
528  |  10:03:45  ->  09:59:00  |  34
529  |  10:03:50  ->  09:59:00  |  34
530  |  10:03:55  ->  09:59:00  |  34
531  |  10:04:00  ->  09:59:00  |  34
532  |  10:04:05  ->  10:00:00  |  35
533  |  10:04:10  ->  10:00:00  |  35
534  |  10:04:15  ->  10:00:00  |  35
535  |  10:04:20  ->  10:00:00  |  35
536  |  10:04:25  ->  10:00:00  |  35
537  |  10:04:30  ->  10:00:00  |  35
538  |  10:04:35  ->  10:00:00  |  35
539  |  10:0

750  |  10:22:15  ->  10:18:00  |  53
751  |  10:22:20  ->  10:18:00  |  53
752  |  10:22:25  ->  10:18:00  |  53
753  |  10:22:30  ->  10:18:00  |  53
754  |  10:22:35  ->  10:18:00  |  53
755  |  10:22:40  ->  10:18:00  |  53
756  |  10:22:45  ->  10:18:00  |  53
757  |  10:22:50  ->  10:18:00  |  53
758  |  10:22:55  ->  10:18:00  |  53
759  |  10:23:00  ->  10:18:00  |  53
760  |  10:23:05  ->  10:19:00  |  54
761  |  10:23:10  ->  10:19:00  |  54
762  |  10:23:15  ->  10:19:00  |  54
763  |  10:23:20  ->  10:19:00  |  54
764  |  10:23:25  ->  10:19:00  |  54
765  |  10:23:30  ->  10:19:00  |  54
766  |  10:23:35  ->  10:19:00  |  54
767  |  10:23:40  ->  10:19:00  |  54
768  |  10:23:45  ->  10:19:00  |  54
769  |  10:23:50  ->  10:19:00  |  54
770  |  10:23:55  ->  10:19:00  |  54
771  |  10:24:00  ->  10:19:00  |  54
772  |  10:24:05  ->  10:20:00  |  55
773  |  10:24:10  ->  10:20:00  |  55
774  |  10:24:15  ->  10:20:00  |  55
775  |  10:24:20  ->  10:20:00  |  55
776  |  10:2

986  |  10:41:55  ->  10:37:00  |  72
987  |  10:42:00  ->  10:37:00  |  72
988  |  10:42:05  ->  10:38:00  |  73
989  |  10:42:10  ->  10:38:00  |  73
990  |  10:42:15  ->  10:38:00  |  73
991  |  10:42:20  ->  10:38:00  |  73
992  |  10:42:25  ->  10:38:00  |  73
993  |  10:42:30  ->  10:38:00  |  73
994  |  10:42:35  ->  10:38:00  |  73
995  |  10:42:40  ->  10:38:00  |  73
996  |  10:42:45  ->  10:38:00  |  73
997  |  10:42:50  ->  10:38:00  |  73
998  |  10:42:55  ->  10:38:00  |  73
999  |  10:43:00  ->  10:38:00  |  73
1000  |  10:43:05  ->  10:39:00  |  74
1001  |  10:43:10  ->  10:39:00  |  74
1002  |  10:43:15  ->  10:39:00  |  74
1003  |  10:43:20  ->  10:39:00  |  74
1004  |  10:43:25  ->  10:39:00  |  74
1005  |  10:43:30  ->  10:39:00  |  74
1006  |  10:43:35  ->  10:39:00  |  74
1007  |  10:43:40  ->  10:39:00  |  74
1008  |  10:43:45  ->  10:39:00  |  74
1009  |  10:43:50  ->  10:39:00  |  74
1010  |  10:43:55  ->  10:39:00  |  74
1011  |  10:44:00  ->  10:39:00  |  74


...

1718  |  13:12:45  ->  13:08:00  |  133
1719  |  13:12:50  ->  13:08:00  |  133
1720  |  13:12:55  ->  13:08:00  |  133
1721  |  13:13:00  ->  13:08:00  |  133
1722  |  13:13:05  ->  13:09:00  |  134
1723  |  13:13:10  ->  13:09:00  |  134
1724  |  13:13:15  ->  13:09:00  |  134
1725  |  13:13:20  ->  13:09:00  |  134
1726  |  13:13:25  ->  13:09:00  |  134
1727  |  13:13:30  ->  13:09:00  |  134
1728  |  13:13:35  ->  13:09:00  |  134
1729  |  13:13:40  ->  13:09:00  |  134
1730  |  13:13:45  ->  13:09:00  |  134
1731  |  13:13:50  ->  13:09:00  |  134
1732  |  13:13:55  ->  13:09:00  |  134
1733  |  13:14:00  ->  13:09:00  |  134
1734  |  13:14:05  ->  13:10:00  |  135
1735  |  13:14:10  ->  13:10:00  |  135
1736  |  13:14:15  ->  13:10:00  |  135
1737  |  13:14:20  ->  13:10:00  |  135
1738  |  13:14:25  ->  13:10:00  |  135
1739  |  13:14:30  ->  13:10:00  |  135
1740  |  13:14:35  ->  13:10:00  |  135
1741  |  13:14:40  ->  13:10:00  |  135
1742  |  13:14:45  ->  13:10:00  |  135


1966  |  13:33:25  ->  13:29:00  |  154
1967  |  13:33:30  ->  13:29:00  |  154
1968  |  13:33:35  ->  13:29:00  |  154
1969  |  13:33:40  ->  13:29:00  |  154
1970  |  13:33:45  ->  13:29:00  |  154
1971  |  13:33:50  ->  13:29:00  |  154
1972  |  13:33:55  ->  13:29:00  |  154
1973  |  13:34:00  ->  13:29:00  |  154
1974  |  13:34:05  ->  13:30:00  |  155
1975  |  13:34:10  ->  13:30:00  |  155
1976  |  13:34:15  ->  13:30:00  |  155
1977  |  13:34:20  ->  13:30:00  |  155
1978  |  13:34:25  ->  13:30:00  |  155
1979  |  13:34:30  ->  13:30:00  |  155
1980  |  13:34:35  ->  13:30:00  |  155
1981  |  13:34:40  ->  13:30:00  |  155
1982  |  13:34:45  ->  13:30:00  |  155
1983  |  13:34:50  ->  13:30:00  |  155
1984  |  13:34:55  ->  13:30:00  |  155
1985  |  13:35:00  ->  13:30:00  |  155
1986  |  13:35:05  ->  13:31:00  |  156
1987  |  13:35:10  ->  13:31:00  |  156
1988  |  13:35:15  ->  13:31:00  |  156
1989  |  13:35:20  ->  13:31:00  |  156
1990  |  13:35:25  ->  13:31:00  |  156


2221  |  13:54:40  ->  13:50:00  |  175
2222  |  13:54:45  ->  13:50:00  |  175
2223  |  13:54:50  ->  13:50:00  |  175
2224  |  13:54:55  ->  13:50:00  |  175
2225  |  13:55:00  ->  13:50:00  |  175
2226  |  13:55:05  ->  13:51:00  |  176
2227  |  13:55:10  ->  13:51:00  |  176
2228  |  13:55:15  ->  13:51:00  |  176
2229  |  13:55:20  ->  13:51:00  |  176
2230  |  13:55:25  ->  13:51:00  |  176
2231  |  13:55:30  ->  13:51:00  |  176
2232  |  13:55:35  ->  13:51:00  |  176
2233  |  13:55:40  ->  13:51:00  |  176
2234  |  13:55:45  ->  13:51:00  |  176
2235  |  13:55:50  ->  13:51:00  |  176
2236  |  13:55:55  ->  13:51:00  |  176
2237  |  13:56:00  ->  13:51:00  |  176
2238  |  13:56:05  ->  13:52:00  |  177
2239  |  13:56:10  ->  13:52:00  |  177
2240  |  13:56:15  ->  13:52:00  |  177
2241  |  13:56:20  ->  13:52:00  |  177
2242  |  13:56:25  ->  13:52:00  |  177
2243  |  13:56:30  ->  13:52:00  |  177
2244  |  13:56:35  ->  13:52:00  |  177
2245  |  13:56:40  ->  13:52:00  |  177


2482  |  14:16:25  ->  14:12:00  |  197
2483  |  14:16:30  ->  14:12:00  |  197
2484  |  14:16:35  ->  14:12:00  |  197
2485  |  14:16:40  ->  14:12:00  |  197
2486  |  14:16:45  ->  14:12:00  |  197
2487  |  14:16:50  ->  14:12:00  |  197
2488  |  14:16:55  ->  14:12:00  |  197
2489  |  14:17:00  ->  14:12:00  |  197
2490  |  14:17:05  ->  14:13:00  |  198
2491  |  14:17:10  ->  14:13:00  |  198
2492  |  14:17:15  ->  14:13:00  |  198
2493  |  14:17:20  ->  14:13:00  |  198
2494  |  14:17:25  ->  14:13:00  |  198
2495  |  14:17:30  ->  14:13:00  |  198
2496  |  14:17:35  ->  14:13:00  |  198
2497  |  14:17:40  ->  14:13:00  |  198
2498  |  14:17:45  ->  14:13:00  |  198
2499  |  14:17:50  ->  14:13:00  |  198
2500  |  14:17:55  ->  14:13:00  |  198
2501  |  14:18:00  ->  14:13:00  |  198
2502  |  14:18:05  ->  14:14:00  |  199
2503  |  14:18:10  ->  14:14:00  |  199
2504  |  14:18:15  ->  14:14:00  |  199
2505  |  14:18:20  ->  14:14:00  |  199
2506  |  14:18:25  ->  14:14:00  |  199


2747  |  14:38:30  ->  14:34:00  |  219
2748  |  14:38:35  ->  14:34:00  |  219
2749  |  14:38:40  ->  14:34:00  |  219
2750  |  14:38:45  ->  14:34:00  |  219
2751  |  14:38:50  ->  14:34:00  |  219
2752  |  14:38:55  ->  14:34:00  |  219
2753  |  14:39:00  ->  14:34:00  |  219
2754  |  14:39:05  ->  14:35:00  |  220
2755  |  14:39:10  ->  14:35:00  |  220
2756  |  14:39:15  ->  14:35:00  |  220
2757  |  14:39:20  ->  14:35:00  |  220
2758  |  14:39:25  ->  14:35:00  |  220
2759  |  14:39:30  ->  14:35:00  |  220
2760  |  14:39:35  ->  14:35:00  |  220
2761  |  14:39:40  ->  14:35:00  |  220
2762  |  14:39:45  ->  14:35:00  |  220
2763  |  14:39:50  ->  14:35:00  |  220
2764  |  14:39:55  ->  14:35:00  |  220
2765  |  14:40:00  ->  14:35:00  |  220
2766  |  14:40:05  ->  14:36:00  |  221
2767  |  14:40:10  ->  14:36:00  |  221
2768  |  14:40:15  ->  14:36:00  |  221
2769  |  14:40:20  ->  14:36:00  |  221
2770  |  14:40:25  ->  14:36:00  |  221
2771  |  14:40:30  ->  14:36:00  |  221


In [ ]:
idx = 2959

In [ ]:
%%timeit
data.get_ma5pm_anchor_idx(idx)

In [ ]:
%%timeit
data.get_time_lapse(idx)

In [ ]:
############################ rebuld from minite data ##############################

In [9]:
from libs.utils import Utils
Utils.update_symbols()

In [10]:
Utils.update_symbols()

In [12]:
q = Quotation()

In [16]:
result = await q.min_data(['002717'])
result

{'002717': {'date': '210823',
  'data': [('0930', 2.78, 4373),
   ('0931', 2.8, 4898),
   ('0932', 2.79, 6269),
   ('0933', 2.8, 7750),
   ('0934', 2.79, 8599),
   ('0935', 2.8, 12354),
   ('0936', 2.8, 12646),
   ('0937', 2.79, 13655),
   ('0938', 2.8, 14061),
   ('0939', 2.8, 14199),
   ('0940', 2.79, 15059),
   ('0941', 2.8, 15217),
   ('0942', 2.8, 15600),
   ('0943', 2.8, 16731),
   ('0944', 2.8, 19524),
   ('0945', 2.81, 21297),
   ('0946', 2.8, 21497),
   ('0947', 2.81, 21683),
   ('0948', 2.81, 24793),
   ('0949', 2.81, 25613),
   ('0950', 2.81, 26462),
   ('0951', 2.81, 27204),
   ('0952', 2.81, 27512),
   ('0953', 2.81, 28871),
   ('0954', 2.81, 28933),
   ('0955', 2.81, 29311),
   ('0956', 2.8, 29595),
   ('0957', 2.81, 30363),
   ('0958', 2.81, 30559),
   ('0959', 2.81, 30795),
   ('1000', 2.8, 31180),
   ('1001', 2.81, 31290),
   ('1002', 2.8, 31653),
   ('1003', 2.81, 31702),
   ('1004', 2.82, 33492),
   ('1005', 2.82, 33709),
   ('1006', 2.81, 33809),
   ('1007', 2.82, 3

In [17]:
result = await q.real(['002717'])
result

{'002717': {'name': '岭南股份',
  'open': 2.78,
  'close': 2.78,
  'now': 2.86,
  'high': 2.88,
  'low': 2.78,
  'buy': 2.85,
  'sell': 2.86,
  'turnover': 13783850,
  'volume': 39001910.75,
  'bid1_volume': 753300,
  'bid1': 2.85,
  'bid2_volume': 386100,
  'bid2': 2.84,
  'bid3_volume': 299300,
  'bid3': 2.83,
  'bid4_volume': 178350,
  'bid4': 2.82,
  'bid5_volume': 200100,
  'bid5': 2.81,
  'ask1_volume': 271110,
  'ask1': 2.86,
  'ask2_volume': 319513,
  'ask2': 2.87,
  'ask3_volume': 472240,
  'ask3': 2.88,
  'ask4_volume': 356300,
  'ask4': 2.89,
  'ask5_volume': 299300,
  'ask5': 2.9,
  'date': '2021-08-23',
  'time': '15:00:03'}}

In [19]:
data.check_points

array([1628730890, 1628730900, 1628730905, ..., 1628751595, 1628751600,
       1628751610], dtype=uint32)

In [39]:
today = time.strftime('%Y-%m-%d')

check_points = []
for start_time in ['09:30:00', '13:00:00']:
    timestamp = int(time.mktime(time.strptime(f'{today} {start_time}', '%Y-%m-%d %H:%M:%S')))
    for i in range(121):
        check_points.append(timestamp+i*60)

In [44]:
symbols = Utils.get_running_symbols()

In [45]:
today = time.strftime('%Y%m%d')
dd = DailyData(date=today, symbols=symbols,
                 check_points=check_points, create=True)

In [53]:
await dd.prepare(symbols, check_points)

In [54]:
########################   测试 get_ma5pm_anchor_idx, get_time_lapse  ####################################
for _, ck in enumerate(dd.check_points):
    print(_, ' | ',
          time.strftime("%H:%M:%S", time.localtime(ck)),' -> ',
          time.strftime("%H:%M:%S", time.localtime(dd.check_points[dd.get_ma5pm_anchor_idx(_)])), ' | ',
          dd.get_time_lapse(_)
         )

0  |  09:30:00  ->  15:00:00  |  1
1  |  09:31:00  ->  15:00:00  |  1
2  |  09:32:00  ->  15:00:00  |  2
3  |  09:33:00  ->  15:00:00  |  3
4  |  09:34:00  ->  15:00:00  |  4
5  |  09:35:00  ->  15:00:00  |  5


IndexError: list index out of range

In [55]:
dd.active_time_blocks

[((0, 1629682200, '2021-08-23 09:30:00'),
  (120, 1629689400, '2021-08-23 11:30:00')),
 ((121, 1629694800, '2021-08-23 13:00:00'),
  (241, 1629702000, '2021-08-23 15:00:00'))]

In [51]:
import types

In [52]:
def post_init(self):
    self.active_time_blocks = []
    self.check_interval = self.check_points[2] - self.check_points[1]
    start_time_0_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 09:15:00'
    start_time_1_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 09:30:00'
    start_time_2_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 13:00:00'
    end_time_0_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 09:25:00'
    end_time_1_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 11:30:00'
    end_time_2_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 15:00:00'
    start_time_0 = int(time.mktime(time.strptime(
        start_time_0_str, '%Y-%m-%d %H:%M:%S')))
    start_time_1 = int(time.mktime(time.strptime(
        start_time_1_str, '%Y-%m-%d %H:%M:%S')))
    start_time_2 = int(time.mktime(time.strptime(
        start_time_2_str, '%Y-%m-%d %H:%M:%S')))
    end_time_0 = int(time.mktime(time.strptime(
        end_time_0_str, '%Y-%m-%d %H:%M:%S')))
    end_time_1 = int(time.mktime(time.strptime(
        end_time_1_str, '%Y-%m-%d %H:%M:%S')))
    end_time_2 = int(time.mktime(time.strptime(
        end_time_2_str, '%Y-%m-%d %H:%M:%S')))

    if start_time_0 >= self.check_points[0]:
        start_time_idx_0 = np.where(
            self.check_points == start_time_0)[0][0]
        end_time_idx_0 = np.where(self.check_points == end_time_0)[0][0]

        self.active_time_blocks.append(
            (
                (start_time_idx_0, start_time_0, start_time_0_str),
                (end_time_idx_0, end_time_0, end_time_0_str)
            )
        )

    start_time_idx_1 = np.where(self.check_points == start_time_1)[0][0]
    end_time_idx_1 = np.where(self.check_points == end_time_1)[0][0]
    self.active_time_blocks.append(
        (
            (start_time_idx_1, start_time_1, start_time_1_str),
            (end_time_idx_1, end_time_1, end_time_1_str)
        )
    )
    start_time_idx_2 = np.where(self.check_points == start_time_2)[0][0]
    end_time_idx_2 = np.where(self.check_points == end_time_2)[0][0]
    self.active_time_blocks.append(
        (
            (start_time_idx_2, start_time_2, start_time_2_str),
            (end_time_idx_2, end_time_2, end_time_2_str)
        )
    )
dd.post_init = types.MethodType(post_init, dd)

In [ ]:
def get_ma5pm_anchor_idx(self, idx):
    ck = self.check_points[idx]
    _ck = ck-300
    
    last_end = None
    for start, end in self.active_time_blocks:
        if _ck < start[1]:
            if start[0] == 0:
                return 0
            elif last_end is None:
                return start[0] - 1
        elif _ck < end[1]:
            result = math.ceil(self.check_points[idx]/60)*60-300
            offset = int((result-self.check_points[idx])/self.check_interval)
            return idx+offset
            
        last_end= end

dd.get_ma5pm_anchor_idx = types.MethodType(get_ma5pm_anchor_idx, dd)